# Example: Let's write a Software Development Kit (SDK) for Polygon.io 
In this example, students will become familiar with RESTful application programming interfaces (API). We'll implement code to work with the [Polygon.io API](https://polygon.io).

* First, look at the [Polygon.io](https://polygon.io) documentation to better understand what endpoints are available, what data is required to make a call, and what we should expect to get back.
* Build a [mutable struct model](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) of each endpoint we want to call. This `struct` should have a field for each piece of information with the same names as the `endpoint` (convention, not required)
* Build a properly formatted [uniform resource locator (URL) string](https://en.wikipedia.org/wiki/URL) containing the information needed by the `endpoint`
* Call the `API` and parse the response using a `handler` function. This takes the raw response coming back from the `API` and turns it into something we can use, e.g., a [DataFrame](https://dataframes.juliadata.org/stable/) for numerical data

## Setup
We use build-in or third-party external packages to read data from [Polygon.io](https://polygon.io/). In the `Include.jl` file, we load these packages, and set some required paths for this example.

In [1]:
include("Include.jl");

  Activating project at `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/Manifest.toml`


As an example use case for configuration files, we store some configuration data (connection information for [Polygon.io](https://polygon.io/)) in the [Configuration.toml file](conf/Configuration.toml).
* We use the [parsefile function](https://docs.julialang.org/en/v1/stdlib/TOML/#TOML.parsefile) exported by the [TOML module in the Julia standard library](https://docs.julialang.org/en/v1/stdlib/TOML/#TOML) to read the configuration file. We store the configuration information in the `configuration` dictionary

In [2]:
configuration = TOML.parsefile(joinpath(_PATH_TO_CONFIG, "Configuration.toml"));

The `baseurl` and `apikey` information is stored in the `connection` section. We can pass multiple `keys` in series to explore the configuration tree and obtain the required data.

In [3]:
baseurl = configuration["connection"]["baseurl"]; # this is short-hand, what is going on?
apikey = configuration["connection"]["apikey"];

## About Polygon.io
[Polygon.io](https://polygon.io) is reimagining financial market data for the 21st century by removing barriers, simplifying access, and creating frictionless, forward-thinking technologies. 
* [Polygon.io](https://polygon.io) believes that access to market data should be a fundamental right, not a privilege reserved for a select few. They provide paid and free data access tiers (even the free access level is excellent).

## Build an endpoint model
In this example, let's build code to call against the [Aggregates Polygon.io Stocks API endpoint](https://polygon.io/docs/stocks/getting-started). We've created the `MyPolygonStocksAggregatesEndpointModel` struct that holds all the data required to make the API call. We build an instance of the `MyPolygonStocksAggregatesEndpointModel` type using a `build(...)` function encoded in `Factory.jl`. 
* The `build(...)` method takes two arguments: first, the type we want to build, and second, the data required to build the `MyPolygonStocksAggregatesEndpointModel.` The data is encoded as a [NamedTuple type](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple). Save the `MyPolygonStocksAggregatesEndpointModel` instance as the `model` variable:

In [4]:
model = build(MyPolygonStocksAggregatesEndpointModel, (
            apikey = apikey, 
            stocksTicker = "AMD", # let's pick something
            multiplier = 1, 
            timespan = "minute", # we can change the time value; see the API documentation
            from = Date(2024,01,12), # start data
            to = Date(2024,01,12), # end date
            adjusted = true, 
            sort = "asc", 
            limit = 5000 # max number of records to return
    ));

## Build the URL to call Polygon.io using a `GET` request
Now that we have specified all the options for the `MyPolygonStocksAggregatesEndpointModel,` let's take this data and encode it into a [URL string](https://en.wikipedia.org/wiki/Query_string) that we'll pass to the `API` as an [HTTP GET request](https://en.wikipedia.org/wiki/HTTP#Request_methods). 
* We do this using a `build(...)` method, passing in the `baseurl` data from the configuration file as the first argument and the model as the second argument. Let's save `URL` in the `endpoint_url` variable

In [5]:
endpoint_url = build(baseurl, model);

## Call the `API` and get the return data formatted as a `DataFrame`
Next, we call the `URL` string (make a `GET` request) and then parse the `response` from the `API.` In this case, the response will return as [JSON encoded data](https://en.wikipedia.org/wiki/JSON).
* For each endpoint, we need to build a specific `handler` function that understands the response data format and extracts the data we care about or returns everything to the caller.
* For the [Stocks Aggregate Polygon.io endpoint](https://polygon.io/docs/stocks/get_v2_aggs_ticker__stocksticker__range__multiplier___timespan___from___to) we want the numerical price and volume data stored in a [DataFrame](https://dataframes.juliadata.org/stable/)
* This calling syntax is `super strange` but is a typical pattern in the `Juliaverse.` What does it seem like is going on? (and then we'll look at what is actually happening)

In [6]:
data = MyPolygonStocksAggregatesEndpointModel(endpoint_url)

"Error: $(result_model |> something)" = "Error: ErrorException(\"HTTP GET call failed with status code 401\\nStacktrace:\\n  [1] _http_get_call_with_url(url::String)\\n    @ Main ~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/src/Network.jl:16\\n  [2] _api(model::Type{MyPolygonStocksAggregatesEndpointModel}, complete_url_string::String; handler::typeof(_default_response_handler))\\n    @ Main ~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/src/Network.jl:36\\n  [3] _api\\n    @ ~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/src/Network.jl:32 [inlined]\\n  [4] MyPolygonStocksAggregatesEndpointModel(baseurl::String)\\n    @ Main ~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/src/Network.jl:52\\n  [5] top-level scope\\n    @ In[6]:1\\n  [6] eval\\n    @ ./boot.jl:385 [inlined]\\n  [7] include_string(mapexpr::typeof(REPL.softscope), mod::Module, code::String, filename::String)\\n    @ Base ./loading.jl:2070\\n  [8] soft

LoadError: MethodError: no method matching _default_response_handler(::Type{MyPolygonStocksAggregatesEndpointModel}, ::Nothing)

[0mClosest candidates are:
[0m  _default_response_handler(::Type{T}, [91m::String[39m) where T<:AbstractPolygonEndpointModel
[0m[90m   @[39m [35mMain[39m [90m~/Desktop/julia_work/CHEME-4800-5800-Examples-AY-2024/week-4/L4a/src/[39m[90m[4mHandlers.jl:41[24m[39m


## Visualize
`Unhide` the code block below to see how we plotted the `vwap,` `open,` and the `close` prices for your ticker of interest. This code uses functions exported by the [Plots.jl package](https://github.com/JuliaPlots/Plots.jl)

In [7]:
plot(data[:,:vw], lw=3, label="VWAP")
plot!(data[:,:o], lw=3, label="open")
plot!(data[:,:c], lw=3, label="close")
xlabel!("Trading period index", fontsize = 18)
ylabel!("Share price (USD/share)", fontsize = 18)

LoadError: UndefVarError: `data` not defined